In [2]:
from os import listdir
from os.path import isfile, join
import os
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import kagglehub

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Activation, MaxPooling1D, Dropout
from tensorflow.keras.utils import plot_model,to_categorical

/Users/katherinebeaty/Documents/DS340 Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Download latest version of the database 
path = kagglehub.dataset_download("vbookshelf/respiratory-sound-database")

print("Path to dataset files:", path)

Path to dataset files: /Users/katherinebeaty/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2


In [31]:
# extract lables for kaggle dataset
# Path to patient diagnosis CSV
csv_path_kaggle = os.path.join(path, "Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv")

# Load CSV
df_kaggle = pd.read_csv(csv_path_kaggle, header=None, names=["Patient_ID", "Diagnosis"])

# Convert to dictionary {Patient_ID: Diagnosis}
patient_labels_kaggle = dict(zip(df_kaggle["Patient_ID"].astype(str), df_kaggle["Diagnosis"]))

# Display first few mappings
print(list(patient_labels_kaggle.items())[:10])  # Check some values

[('101', 'URTI'), ('102', 'Healthy'), ('103', 'Asthma'), ('104', 'COPD'), ('105', 'URTI'), ('106', 'COPD'), ('107', 'COPD'), ('108', 'LRTI'), ('109', 'COPD'), ('110', 'COPD')]


In [33]:
# extract labels for Mendeley dataset
csv_path_mendeley = "audioDataMendeley/Data annotation.csv"

# read csv and only take necessary columns
df_mendeley_raw = pd.read_csv(csv_path_mendeley)
col_index = df_mendeley_raw.columns.get_loc('Diagnosis')
df_mendeley = df_mendeley_raw.iloc[:, :col_index + 1]

# drop heart failure diagnosis
df_mendeley = df_mendeley[~df_mendeley["Diagnosis"].str.lower().str.contains("heart failure", na=False)]

# Change N (normal) to Healthy
df_mendeley["Diagnosis"] = df_mendeley["Diagnosis"].replace({"N": "Healthy"})

# add patient ID column and set as index
df_mendeley.insert(0, "Patient_ID", range(1, len(df_mendeley) + 1))

# formatting for capitalizig diasnosis with expection of acronyms
keep_upper = {"COPD", "BRON"}

def formatting(diagnosis):
    if pd.isna(diagnosis):
        return diagnosis
    text_clean = diagnosis.strip()
    # If the diagnosis (case-insensitive) matches one we want to keep uppercase
    if text_clean.upper() in keep_upper:
        return text_clean.upper()
    else:
        return text_clean.title()

df_mendeley["Diagnosis"] = df_mendeley["Diagnosis"].apply(formatting)

# Convert to dictionary {Patient_ID: Diagnosis}
patient_labels_mendeley = dict(zip(df_mendeley["Patient_ID"].astype(str), df_mendeley["Diagnosis"]))

print(list(patient_labels_mendeley.items())[:10])  # Check some values

[('1', 'Asthma'), ('2', 'Asthma'), ('3', 'Asthma'), ('4', 'Plueral Effusion'), ('5', 'Plueral Effusion'), ('6', 'Asthma'), ('7', 'Asthma'), ('8', 'Asthma'), ('9', 'Asthma'), ('10', 'COPD')]
